Flatten()은 1줄로 만들어서 Nueral Network에 1차원으로 데이터를 넣을 수 있게끔 하는데, 이러한 과정을 거치게 되면, 입력 데이터의 형상이 무너지는 부작용이 있다. 이는 단점으로써 작용할 수 있다.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [0]:
batch_size = 128
num_classes = 10
epochs = 5

In [0]:
# input image dimensions
img_rows, img_cols = 28, 28

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [5]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
                   
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
                   
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 163s 3ms/step - loss: 0.2650 - acc: 0.9189 - val_loss: 0.0598 - val_acc: 0.9800
Epoch 2/5
60000/60000 [==============================] - 164s 3ms/step - loss: 0.0895 - acc: 0.9737 - val_loss: 0.0409 - val_acc: 0.9858
Epoch 3/5
60000/60000 [==============================] - 167s 3ms/step - loss: 0.0643 - acc: 0.9809 - val_loss: 0.0345 - val_acc: 0.9884
Epoch 4/5
60000/60000 [==============================] - 164s 3ms/step - loss: 0.0535 - acc: 0.9837 - val_loss: 0.0337 - val_acc: 0.9889
Epoch 5/5
60000/60000 [==============================] - 164s 3ms/step - loss: 0.0468 - acc: 0.9856 - val_loss: 0.0316 - val_acc: 0.9895


In [9]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.031618887545418695
Test accuracy: 0.9895
